In [1]:
import numpy as np

In [2]:
from pandas import Series, DataFrame

In [3]:
import pandas as pd

In [4]:
from pandas import read_html

In [5]:
url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
url_wb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'

In [6]:
dframe_qb = pd.io.html.read_html(url_qb_season17)
dframe_rb = pd.io.html.read_html(url_rb_season17)
dframe_wb = pd.io.html.read_html(url_wb_season17)
dframe_te = pd.io.html.read_html(url_te_season17)
dframe_def = pd.io.html.read_html(url_def_season17)

In [7]:
qb = dframe_qb[0]

In [8]:
dataMatrix = []

In [9]:
#Get name from Player column and save to nameIndex
nameIndex = []
data = qb['\xa0'].get("Player")
for i in data:
    nameIndex.append(i)
    
dataMatrix.append(nameIndex)
#dataMatrix.append(nameIndex)

In [10]:
bye = []
data = qb['\xa0'].get("Bye")
for i in data:
    bye.append(i)
    
dataMatrix.append(bye)


In [11]:
pts = []
data = qb['\xa0'].get("Pts*")
for i in data:
    pts.append(i)
    
dataMatrix.append(pts)

In [12]:
#Get Passing attempt from Att Data 
passingAtt = []
data = qb['Passing'].get("Att")
for i in data:
    passingAtt.append(i)
    
dataMatrix.append(passingAtt)


In [13]:
passingCmp = []
data = qb['Rushing'].get('Cmp')
for i in data:
    passingCmp.append(i)
    
dataMatrix.append(passingCmp)


In [14]:
passingYds = []
data = qb['Receiving'].get('Yds')
for i in data:
    passingYds.append(i)
    
dataMatrix.append(passingYds)

In [15]:
passingTD = []
data = qb['Fumbles'].get('TD')
for i in data:
    passingTD.append(i)
    
dataMatrix.append(passingTD)

In [16]:
passingInt = []
data = qb['Unnamed: 7_level_0'].get('Int')
for i in data:
    passingInt.append(i)

dataMatrix.append(passingInt)

In [17]:
passing2Pt = []
data = qb['Unnamed: 8_level_0'].get('2Pt')
for i in data:
    passing2Pt.append(i)
    
dataMatrix.append(passing2Pt)

In [18]:
rushingAtt = []
data = qb['Unnamed: 9_level_0'].get('Att')
for i in data:
    rushingAtt.append(i)
    
dataMatrix.append(rushingAtt)

In [19]:
rushingYds = []
data = qb['Unnamed: 10_level_0'].get('Yds')
for i in data:
    rushingYds.append(i)
    
dataMatrix.append(rushingYds)

In [20]:
rushingTD = []
data = qb['Unnamed: 11_level_0'].get('TD')
for i in data:
    rushingTD.append(i)
    
dataMatrix.append(rushingTD)

In [21]:
rushing2Pt = []
data = qb['Unnamed: 12_level_0'].get('2Pt')
for i in data:
    rushing2Pt.append(i)
    
dataMatrix.append(rushing2Pt)

In [22]:
receivingRec = []
data = qb['Unnamed: 13_level_0'].get('Rec')
for i in data:
    receivingRec.append(i)
    
dataMatrix.append(receivingRec)

In [23]:
receivingYds = []
data = qb['Unnamed: 14_level_0'].get('Yds')
for i in data:
    receivingYds.append(i)
    
dataMatrix.append(receivingYds)

In [24]:
receivingTD = []
data = qb['Unnamed: 15_level_0'].get('TD')
for i in data:
    receivingTD.append(i)
    
dataMatrix.append(receivingTD)

In [25]:
receiving2Pt = []
data = qb['Unnamed: 16_level_0'].get('2Pt')
for i in data:
    receiving2Pt.append(i)
    
dataMatrix.append(receiving2Pt)

In [26]:
fumblesFL = []
data = qb['Unnamed: 17_level_0'].get('FL')
for i in data:
    fumblesFL.append(i)
    
dataMatrix.append(fumblesFL)

In [27]:
fumblesTD = []
data = qb['Unnamed: 18_level_0'].get('TD')
for i in data:
    fumblesTD.append(i)
    
dataMatrix.append(fumblesTD)

In [28]:
dataMatrix

dataMatrix = np.array(dataMatrix)
dataMatrix = dataMatrix.transpose()

In [29]:

#create a tuple for multi-indexed Dataframe
tup = [('', 'Player'),
       ('', 'Bye'),
       ('', 'Pts'),
       ('Passing', 'Att'),
       ('Passing', 'Cmp'),
       ('Passing', 'Yds'),
       ('Passing', 'TD'),
       ('Passing', 'Int'),
       ('Passing', '2Pt'),
       ('Rushing', 'Att'),
       ('Rushing', 'Yds'),
       ('Rushing', 'TD'),
       ('Rushing', '2Pt'),
       ('Receiving', 'Rec'),
       ('Receiving', 'Yds'),
       ('Receiving', 'TD'),
       ('Receiving', '2Pt'),
       ('Fumbles', 'FL'),
       ('Fumbles', 'TD')
      ]



In [30]:
index = pd.MultiIndex.from_tuples(tup)
qb.index

RangeIndex(start=0, stop=73, step=1)

In [36]:
dframe1 = DataFrame(dataMatrix, columns = index)
dframe1

Passing                     \
                                 Player Bye    Pts     Att  Cmp   Yds  TD Int   
0               Russell WilsonR. Wilson   6  402.0     553  339  3983  34  11   
1                     Tom BradyT. Brady   9  352.0     581  385  4577  32   8   
2                  Carson WentzC. Wentz  10  338.0     440  265  3296  33   7   
3                    Alex SmithA. Smith  10  334.0     505  341  4042  26   5   
4                   Cam NewtonC. Newton  11  331.0     492  291  3302  22  16   
5           Matthew StaffordM. Stafford   7  326.0     565  371  4446  29  10   
6                Kirk CousinsK. Cousins   0  322.0     540  347  4093  27  13   
7                Philip RiversP. Rivers   9  320.0     575  360  4515  28  10   
8   Ben RoethlisbergerB. Roethlisberger   9  310.0     561  360  4251  28  14   
9                     Jared GoffJ. Goff   8  305.0     477  296  3804  28   7   
10                   Drew BreesD. Brees   0  298.0     536  386  4334  23   8   
11              Dak PrescottD. Prescott   6  295.0     490  308  3324  22  13   
12              Blake BortlesB. Bortles   8  280.0     523  315  3687  21  13   
13                 Case KeenumC. Keenum   9  274.0     481  325  3547  22   7   
14                     Matt RyanM. Ryan   0  256.0     529  342  4095  20  12   
15                 Andy DaltonA. Dalton   6  252.0     496  297  3320  25  12   
16                    Derek CarrD. Carr  10  244.0     515  323  3496  22  13   
17                Tyrod TaylorT. Taylor   6  239.0     420  263  2799  14   4   
18             Jameis WinstonJ. Winston   1  236.0     442  282  3504  19  11   
19                 Josh McCownJ. McCown  11  235.0     397  267  2926  18   9   
20             Marcus MariotaM. Mariota   8  226.0     453  281  3232  13  15   
21                Eli ManningE. Manning   8  223.0     571  352  3468  19  13   
22           Jacoby BrissettJ. Brissett  11  220.0     469  276  3098  13   7   
23                  Joe FlaccoJ. Flacco  10  212.0     549  352  3141  18  13   
24              Deshaun WatsonD. Watson   7  204.0     204  126  1699  19   8   
25                DeShone KizerD. Kizer   9  188.0     476  255  2894  11  22   
26                  Jay CutlerJ. Cutler   1  186.0     429  266  2666  19  14   
27              Aaron RodgersA. Rodgers   8  157.0     238  154  1675  16   6   
28             Trevor SiemianT. Siemian   0  143.0     349  206  2285  12  14   
29         Mitchell TrubiskyM. Trubisky   9  142.0     330  196  2193   7   7   
..                                  ...  ..    ...     ...  ...   ...  ..  ..   
43                   T.J. YatesT. Yates   7   41.0      97   47   523   4   3   
44                  Kevin HoganK. Hogan   9   38.0      75   46   517   4   5   
45              Sam BradfordS. Bradford   9   32.0      43   32   382   3   0   
46                  Bryce PettyB. Petty  11   24.0     112   55   544   1   3   
47                  David FalesD. Fales   1   20.0      43   29   265   1   1   
48                 Paxton LynchP. Lynch   0   18.0      45   30   295   2   3   
49                 E.J. ManuelE. Manuel  10   15.0      43   24   265   1   1   
50                   Geno SmithG. Smith   8   13.0      36   21   212   1   0   
51               Ryan MallettR. Mallett  10   13.0      16    9    56   2   0   
52           Nathan PetermanN. Peterman   6   12.0      49   24   252   2   5   
53                 Landry JonesL. Jones   9   12.0      28   23   239   1   1   
54            Patrick MahomesP. Mahomes  10   10.0      35   22   284   0   1   
55               Nate SudfeldN. Sudfeld  10    7.0      23   19   134   0   0   
56               Sean MannionS. Mannion   8    5.0      37   22   185   0   0   
57                 Matt CasselM. Cassel   8    5.0      42   25   162   1   2   
58                      Joe WebbJ. Webb   6    3.0       7    2    35   0   1   
59               Cody KesslerC. Kessler   9    2.0      23   11   126   0   1   
6